In [1]:
import datetime
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="darkgrid")
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示
from zipfile import ZipFile


In [2]:
import pickle
usingSaved=False


In [4]:
#读取item
if not usingSaved:
    output = open('H:\\pythonchengx_u\\Tianchiantai\\dianshangtuijian\\item_attr.pkl', 'wb')
    item_attr=pd.read_csv('H:\\pythonchengx_u\\Tianchiantai\\dianshangtuijian\\Antai_AE_round1_item_attr_20190626.csv')

    pickle.dump(item_attr, output)
    output.close()
else:
    item_attr = pickle.load(open('H:\\pythonchengx_u\\Tianchiantai\\dianshangtuijian\\item_attr.pkl', 'rb'), encoding='iso-8859-1')
#读取train  
if not usingSaved:
    output = open('H:\\pythonchengx_u\\Tianchiantai\\dianshangtuijian\\train.pkl', 'wb')

    train=pd.read_csv('H:\\pythonchengx_u\\Tianchiantai\\dianshangtuijian\\Antai_AE_round1_train_20190626.csv')
#     f.close()
#     myzip.close()
    train['create_order_time'] = train.create_order_time.apply(lambda x:pd.to_datetime(x))
    train['hour']=train['create_order_time'].dt.hour
    train['date']=train['create_order_time'].dt.day
    train['month']=train['create_order_time'].dt.month
    train['year']=train['create_order_time'].dt.year
    train['month-date'] = train.month.astype(str)+'-'+train.date.astype(str)
    train['count'] = 1
    train['dayofweek']=train['create_order_time'].dt.dayofweek
    train['isweekend']=train['dayofweek'].apply(lambda x:0 if x<5 else 1)
    pickle.dump(train, output)
    output.close()
else:
    train = pickle.load(open('H:\\pythonchengx_u\\Tianchiantai\\dianshangtuijian\\train.pkl', 'rb'),encoding='iso-8859-1')
#读取test
if not usingSaved:
    output = open('H:\\pythonchengx_u\\Tianchiantai\\dianshangtuijian\\test.pkl', 'wb')
    test=pd.read_csv('H:\\pythonchengx_u\\Tianchiantai\\dianshangtuijian\\Antai_AE_round1_test_20190626.csv')
    test['create_order_time'] = test.create_order_time.apply(lambda x:pd.to_datetime(x))
    test['hour']=test['create_order_time'].dt.hour
    test['date']=test['create_order_time'].dt.day
    test['month']=test['create_order_time'].dt.month
    test['year']=test['create_order_time'].dt.year
    test['month-date'] = test.month.astype(str)+'-'+test.date.astype(str)
    test['count'] = 1
    train['dayofweek']=train['create_order_time'].dt.dayofweek
    train['isweekend']=train['dayofweek'].apply(lambda x:0 if x<5 else 1)
    pickle.dump(test, output)
    output.close()
else:
    test = pickle.load(open('H:\\pythonchengx_u\\Tianchiantai\\dianshangtuijian\\test.pkl', 'rb'), encoding='iso-8859-1')

path ='H:/pythonchengx_u/Tianchiantai/'
item = pd.read_csv('H:\\pythonchengx_u\\Tianchiantai\\dianshangtuijian\\Antai_AE_round1_item_attr_20190626.csv')
train = pd.read_csv('H:\\pythonchengx_u\\Tianchiantai\\dianshangtuijian\\Antai_AE_round1_train_20190626.csv')
test = pd.read_csv('H:\\pythonchengx_u\\Tianchiantai\\dianshangtuijian\\Antai_AE_round1_test_20190626.csv')

df = pd.concat([train.assign(is_train=1), test.assign(is_train=0)])

df['create_order_time'] = pd.to_datetime(df['create_order_time'])
df['date'] = df['create_order_time'].dt.date
df['day'] = df['create_order_time'].dt.day
df['hour'] = df['create_order_time'].dt.hour

df = pd.merge(df, item, how='left', on='item_id')



In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tfcf.metrics import mae  # 需要安装这个包；最上面有link
from tfcf.metrics import rmse
from tfcf.datasets import ml1m
from tfcf.config import Config
from tfcf.models.svd import SVD
# from svd import SVD
from tfcf.models.svdpp import SVDPP
from sklearn.model_selection import train_test_split
from zipfile import ZipFile

import numpy as np
import tensorflow as tf

try:
    from tensorflow.keras import utils
except:
    from tensorflow.contrib.keras import utils
from tfcf.models.model_base import BaseModel
from tfcf.utils.data_utils import BatchGenerator
from tfcf.metrics import mae
from tfcf.metrics import rmse

class SVD(BaseModel):
    """Collaborative filtering model based on SVD algorithm.
    """

    def __init__(self, config, sess):
        super(SVD, self).__init__(config)
        self._sess = sess

    def _create_placeholders(self):
        """Returns the placeholders.
        """
        with tf.variable_scope('placeholder'):
            users = tf.placeholder(tf.int32, shape=[None, ], name='users')
            items = tf.placeholder(tf.int32, shape=[None, ], name='items')
            ratings = tf.placeholder(
                tf.float32, shape=[None, ], name='ratings')

        return users, items, ratings

    def _create_constants(self, mu):
        """Returns the constants.
        """
        with tf.variable_scope('constant'):
            _mu = tf.constant(mu, shape=[], dtype=tf.float32)

        return _mu

    def _create_user_terms(self, users):
        """Returns the tensors related to users.
        """
        num_users = self.num_users
        num_factors = self.num_factors

        with tf.variable_scope('user',reuse=tf.AUTO_REUSE):
            self.user_embeddings = tf.get_variable(
                name='embedding',
                shape=[num_users, num_factors],
                initializer=tf.contrib.layers.xavier_initializer(),
                regularizer=tf.contrib.layers.l2_regularizer(self.reg_p_u))

            self.user_bias = tf.get_variable(
                name='bias',
                shape=[num_users, ],
                initializer=tf.contrib.layers.xavier_initializer(),
                regularizer=tf.contrib.layers.l2_regularizer(self.reg_b_u))

            p_u = tf.nn.embedding_lookup(
                self.user_embeddings,
                users,
                name='p_u')

            b_u = tf.nn.embedding_lookup(
                self.user_bias,
                users,
                name='b_u')

        return p_u, b_u

    def _create_item_terms(self, items):
        """Returns the tensors related to items.
        """
        num_items = self.num_items
        num_factors = self.num_factors

        with tf.variable_scope('item',reuse=tf.AUTO_REUSE):
            self.item_embeddings = tf.get_variable(
                name='embedding',
                shape=[num_items, num_factors],
                initializer=tf.contrib.layers.xavier_initializer(),
                regularizer=tf.contrib.layers.l2_regularizer(self.reg_q_i))

            self.item_bias = tf.get_variable(
                name='bias',
                shape=[num_items, ],
                initializer=tf.contrib.layers.xavier_initializer(),
                regularizer=tf.contrib.layers.l2_regularizer(self.reg_b_i))

            q_i = tf.nn.embedding_lookup(
                self.item_embeddings,
                items,
                name='q_i')

            b_i = tf.nn.embedding_lookup(
                self.item_bias,
                items,
                name='b_i')

        return q_i, b_i
    def _get_item_terms(self):
        """Returns the tensors related to items.
        """
        import pickle
        saver = tf.train.Saver()
        item_embeddings = self.item_embeddings.eval()
        saver_path = saver.save(self._sess, './')
        print("saver path: ",saver_path)
        with open('./item_embeddings.pkl', 'wb') as fw:
            pickle.dump({'embeddings': item_embeddings}, fw)

    def _create_prediction(self, mu, b_u, b_i, p_u, q_i):
        """Returns the tensor of prediction.

           Note that the prediction 
            r_hat = \mu + b_u + b_i + p_u * q_i
        """
        with tf.variable_scope('prediction'):
            pred = tf.reduce_sum(
                tf.multiply(p_u, q_i),
                axis=1)

            pred = tf.add_n([b_u, b_i, pred])

            pred = tf.add(pred, mu, name='pred')

        return pred

    def _create_loss(self, pred, ratings):
        """Returns the L2 loss of the difference between
            ground truths and predictions.

           The formula is here:
            L2 = sum((r - r_hat) ** 2) / 2
        """
        with tf.variable_scope('loss'):
            loss = tf.nn.l2_loss(tf.subtract(ratings, pred), name='loss')

        return loss

    def _create_optimizer(self, loss):
        """Returns the optimizer.

           The objective function is defined as the sum of
            loss and regularizers' losses.
        """
        with tf.variable_scope('optimizer'):
            objective = tf.add(
                loss,
                tf.add_n(tf.get_collection(
                    tf.GraphKeys.REGULARIZATION_LOSSES)),
                name='objective')

            try:
                optimizer = tf.contrib.keras.optimizers.Nadam(
                ).minimize(objective, name='optimizer')
            except:
                optimizer = tf.train.AdamOptimizer().minimize(objective, name='optimizer')

        return optimizer

    def _build_graph(self, mu):
        _mu = self._create_constants(mu)

        self._users, self._items, self._ratings = self._create_placeholders()

        p_u, b_u = self._create_user_terms(self._users)
        q_i, b_i = self._create_item_terms(self._items)

        self._pred = self._create_prediction(_mu, b_u, b_i, p_u, q_i)

        loss = self._create_loss(self._ratings, self._pred)

        self._optimizer = self._create_optimizer(loss)

        self._built = True

    def _run_train(self, x, y, epochs, batch_size, validation_data):
        train_gen = BatchGenerator(x, y, batch_size)
        steps_per_epoch = np.ceil(train_gen.length / batch_size).astype(int)

        self._sess.run(tf.global_variables_initializer())

        for e in range(1, epochs + 1):
            print('Epoch {}/{}'.format(e, epochs))

            pbar = utils.Progbar(steps_per_epoch)

            for step, batch in enumerate(train_gen.next(), 1):
                users = batch[0][:, 0]
                items = batch[0][:, 1]
                ratings = batch[1]

                self._sess.run(
                    self._optimizer,
                    feed_dict={
                        self._users: users,
                        self._items: items,
                        self._ratings: ratings
                    })

                pred = self.predict(batch[0])

                update_values = [
                    ('rmse', rmse(ratings, pred)),
                    ('mae', mae(ratings, pred))
                ]

                if validation_data is not None and step == steps_per_epoch:
                    valid_x, valid_y = validation_data
                    valid_pred = self.predict(valid_x)

                    update_values += [
                        ('val_rmse', rmse(valid_y, valid_pred)),
                        ('val_mae', mae(valid_y, valid_pred))
                    ]

                pbar.update(step, values=update_values)

    def train(self, x, y, epochs=100, batch_size=1024, validation_data=None):

        if x.shape[0] != y.shape[0] or x.shape[1] != 2:
            raise ValueError('The shape of x should be (samples, 2) and '
                             'the shape of y should be (samples, 1).')

        if not self._built:
            self._build_graph(np.mean(y))

        self._run_train(x, y, epochs, batch_size, validation_data)
        self._get_item_terms()

    def predict(self, x):
        if not self._built:
            raise RunTimeError('The model must be trained '
                               'before prediction.')

        if x.shape[1] != 2:
            raise ValueError('The shape of x should be '
                             '(samples, 2)')

        pred = self._sess.run(
            self._pred,
            feed_dict={
                self._users: x[:, 0],
                self._items: x[:, 1]
            })

        pred = pred.clip(min=self.min_value, max=self.max_value)

        return pred

In [9]:
train_counts = train.groupby(['buyer_admin_id','item_id'])['count'].sum().reset_index()



In [10]:
train_counts['count_ratio']=train_counts['count']/1271.0

In [90]:
train_counts.item.head()

0       476
1     40488
2     80921
3    192548
4    212912
Name: item, dtype: int64

In [11]:
train_counts.head()

,buyer_admin_id,item_id,count,count_ratio
0,1,1363,1,0.000787
1,1,149829,1,0.000787
2,1,311727,1,0.000787
3,1,837168,1,0.000787
4,1,928511,1,0.000787


In [12]:
train_counts.buyer_admin_id.min(),train_counts.buyer_admin_id.max() # 次数对不上embedding，使用labelencoder规范化



(1, 13046721)

In [13]:
from sklearn import preprocessing
le_buyer = preprocessing.LabelEncoder()
le_buyer.fit(train_counts.buyer_admin_id.unique()) 
le_item = preprocessing.LabelEncoder()
le_item.fit(train_counts.item_id.unique()) 

LabelEncoder()

In [14]:
train_counts['buyer'] = le_buyer.transform(train_counts.buyer_admin_id.values)
train_counts['item'] = le_item.transform(train_counts.item_id.values)

In [15]:
train_counts['buyer'] = train_counts['buyer']+1
train_counts['item'] = train_counts['item']+1

In [16]:
train_counts.head()



,buyer_admin_id,item_id,count,count_ratio,buyer,item
0,1,1363,1,0.000787,1,476
1,1,149829,1,0.000787,1,40488
2,1,311727,1,0.000787,1,80921
3,1,837168,1,0.000787,1,192548
4,1,928511,1,0.000787,1,212912


In [17]:

x,y=train_counts[['buyer','item']].values,train_counts.count_ratio.values

In [18]:
# Note that x is a 2D numpy array, 
# x[i, :] contains the user-item pair, and y[i] is the corresponding rating.
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=0)

config = Config()
config.num_users = np.max(x[:, 0]) + 1
config.num_items = np.max(x[:, 1]) + 1
config.min_value = np.min(y)
config.max_value = np.max(y)

with tf.Session() as sess:
    # For SVD++ algorithm, if `dual` is True, then the dual term of items' 
    # implicit feedback will be added into the original SVD++ algorithm.
    # model = SVDPP(config, sess, dual=False)
    # model = SVDPP(config, sess, dual=True)
    model = SVD(config, sess)# 给模型加了保存embedding的一个功能
    model.train(x_train, y_train, validation_data=(
        x_test, y_test), epochs=2, batch_size=8092)
        
    y_pred = model.predict(x_test)
    print('rmse: {}, mae: {}'.format(rmse(y_test, y_pred), mae(y_test, y_pred)))
        
    # Save model
    model.save_model('model/')

Epoch 1/2
1013/1013 [==============================] - 1428s 1s/step - rmse: 0.0049 - mae: 0.0024 - val_rmse: 0.0069 - val_mae: 0.0036
Epoch 2/2
1013/1013 [==============================] - 1428s 1s/step - rmse: 0.0060 - mae: 0.0032 - val_rmse: 0.0069 - val_mae: 0.0036
saver path:  ./
rmse: 0.006903757755839459, mae: 0.0036497351114366185


In [19]:

import pickle
with open('./item_embeddings.pkl', 'rb') as fr:
    data = pickle.load(fr)
    final_embeddings = data['embeddings']

In [20]:
test_counts = test[test.item_id.isin(train.item_id.unique())].groupby(['buyer_admin_id','item_id']).agg({'count':'sum','irank':'max'}).reset_index()
test_counts['item'] = le_item.transform(test_counts.item_id.values)



In [92]:
le_item.classes_

array([       1,        2,        3, ..., 13046731, 13046732, 13046734],
      dtype=int64)

In [100]:
2812048 in le_item.classes_

False

In [21]:
test_counts.head()



,buyer_admin_id,item_id,count,irank,item
0,152,7937154,1,3,1732802
1,152,8410857,1,2,1833065
2,152,8472223,1,4,1846548
3,152,9891513,1,6,2163974
4,282,688799,1,10,159923


In [22]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=31, algorithm='ball_tree').fit(final_embeddings)



In [23]:
test_final_embeddings=[]
for line in test_counts.iterrows():
    test_final_embeddings.append(final_embeddings[line[1]['item']])



In [93]:
len(test_final_embeddings)



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [95]:
type(test_final_embeddings)
test_final_embeddings[:10]

[array([ 3.6345510e-12, -6.8071779e-05, -4.6813291e-13,  9.3123381e-06,
        -9.1887989e-15, -7.1985148e-17,  1.2939977e-16, -6.3635309e-16,
         1.8443060e-16, -1.1711151e-13, -1.7093253e-12,  1.1151691e-05,
        -7.3523630e-08,  3.3653441e-11,  7.4997642e-16], dtype=float32),
 array([-1.2819355e-10,  3.6641419e-11, -2.5750697e-11, -5.8679386e-14,
         4.4789304e-05,  5.0304825e-06, -9.5915099e-17,  3.4231175e-12,
        -1.4824520e-06,  6.3768999e-12, -2.0752393e-12,  2.9699114e-16,
        -2.6310869e-14,  3.8532511e-15, -6.1889313e-12], dtype=float32),
 array([-2.4423443e-23,  5.1311159e-05,  1.2703571e-24,  2.7333766e-07,
         7.6229524e-05,  9.7661097e-09,  5.3876661e-06, -1.6043981e-04,
         1.8425251e-22, -2.4090177e-05,  9.1434759e-07, -7.8082583e-22,
         7.4634554e-10,  4.4656840e-06,  6.6874756e-23], dtype=float32),
 array([ 9.8731753e-15,  1.2809082e-31,  6.0628655e-12,  3.0636801e-31,
        -8.4799137e-09, -6.9859812e-12,  1.5666240e-06, -2.45

In [25]:
distances, indices = nbrs.kneighbors(test_final_embeddings) # 已经按照距离排好序了



In [42]:
distances[0:2,:5]

array([[0.00000000e+00, 6.15640373e-06, 8.74560345e-06, 9.39715264e-06,
        9.85903115e-06],
       [0.00000000e+00, 1.91458147e-06, 2.44983091e-06, 2.71428192e-06,
        2.86048294e-06]])

adistance=distances[:,1:]

adistance[0:2,:5]

np.shape(adistance)

In [46]:
indices[:2,:10]

array([[1732802, 1736892, 2581220, 2595460, 1074171, 2671349,  469027,
        1318462,  177631, 1625972],
       [1833065,   41030, 2171572, 2442847, 2622516, 1870332, 1205049,
        1308066, 1637060, 2329768]], dtype=int64)

In [96]:
len(indices)

102848

In [129]:
np.shape(indices)

(102848, 31)

In [77]:
rindices=indices.reshape(-1,1)
rindices[0:10]

array([[1732802],
       [1736892],
       [2581220],
       [2595460],
       [1074171],
       [2671349],
       [ 469027],
       [1318462],
       [ 177631],
       [1625972]], dtype=int64)

True

AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [67]:
np.shape(rindices)

(3188288, 1)

In [52]:
type(indices)

numpy.ndarray

In [83]:
cindices=rindices


In [116]:
type(cindices)
# shape(cindices)
cindices[0:10]

[[1732802],
 [1736892],
 [2581220],
 [2595460],
 [1074171],
 [2671349],
 [469027],
 [1318462],
 [177631],
 [1625972]]

In [120]:
cindices.count([2812048])
cindices.index([2812048])

2843086

In [121]:
cindices[2843086]=[1732802]


In [122]:
[2812048] in cindices

False

In [124]:
dindices=np.array(cindices)

In [125]:
dindices[0:10]

array([[1732802],
       [1736892],
       [2581220],
       [2595460],
       [1074171],
       [2671349],
       [ 469027],
       [1318462],
       [ 177631],
       [1625972]])

In [105]:
type(dindices)

numpy.ndarray

[[1732802]
 [1736892]
 [2581220]
 [2595460]
 [1074171]
 [2671349]
 [ 469027]
 [1318462]
 [ 177631]
 [1625972]]


In [126]:

indices_inversed = le_item.inverse_transform(dindices)



In [127]:
print(indices_inversed[0:20])

[ 7937154  7956777 11837248 11890014  4923358 12208199  2147287  5998605
   770121  7427997  8356883 10056453  2892645  1502966 11112748 11918292
  2369570  7343746   776376  3760863]


In [130]:
indices_inversed_reshape=indices_inversed.reshape(102848, 31)

In [132]:
indices_inversed_reshape[0:2,:]

array([[ 7937154,  7956777, 11837248, 11890014,  4923358, 12208199,
         2147287,  5998605,   770121,  7427997,  8356883, 10056453,
         2892645,  1502966, 11112748, 11918292,  2369570,  7343746,
          776376,  3760863,  4394063,  7062908, 12622706, 11680441,
         6591760,  6185471,  8617073,  9163804,  5108875,  2112375,
        10454008],
       [ 8410857,   151894,  9927200, 11246450, 12006552,  8579541,
         5503412,  5950891,  7476953, 10706841,  8018463,  8449727,
        10838401,  9251342,  5944228,  9662371, 12059293,  1529535,
         2208496, 11931916, 11634281,  8470060, 12134318,   216363,
         4291315,  6993531,  9050405,  5132622, 12304697,  7541396,
         4916669]], dtype=int64)

In [133]:
indices_inversed_str = [str(li) for li in indices_inversed_reshape]



In [135]:
indices_inversed_reshape[:,0]



array([ 7937154,  8410857,  8472223, ...,  9776214, 10391871,  5619538],
      dtype=int64)

array([ 2.62690840e-12,  3.43209096e-12,  1.61058245e-11,  1.61578262e-11,
       -6.41683361e-12,  1.67832858e-11, -1.49542545e-11, -2.99289933e-12,
       -1.93007644e-11,  1.74257233e-12,  4.28644159e-12,  9.68970385e-12,
       -1.30595391e-11, -1.71947923e-11,  1.29671134e-11,  1.58760708e-11,
       -1.43645125e-11,  1.64555387e-12, -1.93583447e-11, -1.04817324e-11,
       -8.90061071e-12,  8.77256456e-13,  1.84948317e-11,  1.48232516e-11,
       -7.95892983e-13, -2.24424467e-12,  4.83694367e-12,  7.45384313e-12,
       -6.54903529e-12, -1.51746554e-11,  1.11691964e-11])

In [137]:
for i in range(indices_inversed_reshape.shape[1]):
    test_counts['recomend_'+str(i)] = indices_inversed_reshape[:,i]



In [138]:
test_counts.head()

,buyer_admin_id,item_id,count,irank,item,recomend_0,recomend_1,recomend_2,recomend_3,recomend_4,...,recomend_21,recomend_22,recomend_23,recomend_24,recomend_25,recomend_26,recomend_27,recomend_28,recomend_29,recomend_30
0,152,7937154,1,3,1732802,7937154,7956777,11837248,11890014,4923358,...,7062908,12622706,11680441,6591760,6185471,8617073,9163804,5108875,2112375,10454008
1,152,8410857,1,2,1833065,8410857,151894,9927200,11246450,12006552,...,8470060,12134318,216363,4291315,6993531,9050405,5132622,12304697,7541396,4916669
2,152,8472223,1,4,1846548,8472223,22396,9812496,11571407,3798258,...,6823781,1109750,4374706,1288234,8364638,11131685,10626091,1717757,4332301,12582560
3,152,9891513,1,6,2163974,9891513,695019,8794781,2494499,2321876,...,6119131,12578155,4981824,12546505,253916,5243040,3427003,53679,4163715,11869533
4,282,688799,1,10,159923,688799,3360568,5660546,6885073,10526709,...,576598,10126081,4618938,656983,3429353,3300782,7810370,4624789,10468371,7936754


In [139]:
test_counts.to_csv('test_counts.csv',index=False,encoding='gbk')

In [140]:
test_counts = pd.read_csv('test_counts.csv')

In [141]:
test_counts.head()



,buyer_admin_id,item_id,count,irank,item,recomend_0,recomend_1,recomend_2,recomend_3,recomend_4,...,recomend_21,recomend_22,recomend_23,recomend_24,recomend_25,recomend_26,recomend_27,recomend_28,recomend_29,recomend_30
0,152,7937154,1,3,1732802,7937154,7956777,11837248,11890014,4923358,...,7062908,12622706,11680441,6591760,6185471,8617073,9163804,5108875,2112375,10454008
1,152,8410857,1,2,1833065,8410857,151894,9927200,11246450,12006552,...,8470060,12134318,216363,4291315,6993531,9050405,5132622,12304697,7541396,4916669
2,152,8472223,1,4,1846548,8472223,22396,9812496,11571407,3798258,...,6823781,1109750,4374706,1288234,8364638,11131685,10626091,1717757,4332301,12582560
3,152,9891513,1,6,2163974,9891513,695019,8794781,2494499,2321876,...,6119131,12578155,4981824,12546505,253916,5243040,3427003,53679,4163715,11869533
4,282,688799,1,10,159923,688799,3360568,5660546,6885073,10526709,...,576598,10126081,4618938,656983,3429353,3300782,7810370,4624789,10468371,7936754


In [143]:
test_counts.groupby('buyer_admin_id').apply(lambda t: t[t.irank==t.irank.min()]).reset_index(drop=True).drop(['item_id','count','irank','item','recomend_0'],axis=1).to_csv('username_svd.csv',index=False,header=False,encoding='gbk')


In [147]:
username_svd = pd.read_csv('username_svd.csv')

In [148]:
username_svd.head()



,152,151894,9927200,11246450,12006552,8579541,5503412,5950891,7476953,10706841,...,8470060,12134318,216363,4291315,6993531,9050405,5132622,12304697,7541396,4916669
0,282,10181555,3046277,9276603,11105688,5955714,12256939,5199140,12509246,11170698,...,5123645,10278770,1744469,1629541,1299826,4797461,3287829,2537097,8998883,3084757
1,321,12461938,5768846,7327574,11561711,4323750,3888946,11467384,2731903,11191705,...,6082751,8680259,1791588,5938961,11925690,2069989,8301436,8714418,3489275,7521686
2,809,7724613,11556680,357107,10109898,9885048,1095495,6642028,6972112,3604449,...,5679146,9532882,216997,8500199,857203,6278132,9862680,4158135,3417109,8708876
3,870,1277862,6162280,11161118,8329694,12210483,4694917,1302351,354222,6083050,...,2143334,12130779,1285791,6552337,3592085,1615696,5210881,170,10441831,5543564
4,2410,5142695,10393629,5902911,2062876,9283588,10430395,4114089,12069164,583972,...,7580015,1371409,5731300,6370676,977100,3891447,5531939,9262785,4996548,8915895


In [149]:
np.shape(username_svd)

(11388, 31)

In [155]:
import copyreg, copy, pickle
import dill
filename= 'H:\\pythonchengx_u\\Tianchiantai\\dianshangtuijian\\SVDglobalsave.pkl'
dill.dump_session(filename)

TypeError: can't pickle SwigPyObject objects